#### Description

In [1]:
print('Author: Leo Pauly (cnlp@leeds.ac.uk) & Nick Wilson (n.wilson@lubs.leeds.ac.uk)')
print('Description: Autmatic database update')

#### Usage intructions & Other info

1. Change/Add User (user variable options: 'leo','nick')
2. Assumed that new entry directory names always start with UKLTD_R_ or UKLTD_W_
3. File processed_dir_list_AC05.txt contains the processed directory list
4. Prefefably install python using anacodna (all in one installation): https://www.anaconda.com/products/individual#windows
5. Run this if 'pyarrow' module is missing':  !pip install pyarrow 
6. Check base_dir variable
7. Databases are stored in the directory (Create one if missing) : UKLTD_Database 
8. Run this if 'patool' module is missing':  !pip install patool pyunpack
9. Delete UKLTD_Database/AC05* directory and processed_dir_list_AC05.txt file (in UKLTD_Scripts dir) when running for the very first time

In [2]:
#!pip install pyarrow 
#!pip install patool pyunpack
#print('Extra modules installed')

#### Imports

In [3]:
import os
import sys
import pandas as pd
import numpy as np
import pyreadstat
from zipfile import ZipFile
import pyunpack
import multiprocessing
import dask.dataframe as dd
import time
import dask
import csv
from dask.diagnostics import ProgressBar
from dask.distributed import Client
dask.config.set(scheduler='threads')

print('Python version:',sys.version)
num_processes = multiprocessing.cpu_count()
print('No: of logical CPU cores available:',num_processes)

AC05_header_dtypes={'AC05': 'str','REGNUM': 'str','INSU001': 'str','INSU002': 'str','INSU003': 'str','INSU004': 'str','INSU005': 'str','INSU006': 'float64','INSU007': 'float64','INSU008': 'float64','INSU009': 'float64','INSU010': 'float64','INSU011': 'float64','INSU012': 'float64','INSU013': 'float64','INSU014': 'float64','INSU015': 'float64','INSU016': 'float64','INSU017': 'float64','INSU018': 'float64','INSU019': 'float64','INSU020': 'float64','INSU021': 'float64','INSU022': 'float64','INSU023': 'float64','INSU024': 'float64','INSU025': 'float64','INSU026': 'float64','INSU027': 'float64','INSU028': 'float64','INSU029': 'float64','INSU030': 'float64','INSU031': 'float64','INSU032': 'float64','INSU033': 'float64','INSU034': 'float64','INSU035': 'float64','INSU036': 'float64','INSU037': 'float64','INSU038': 'float64','INSU039': 'float64','INSU040': 'float64','INSU041': 'float64','INSU042': 'float64','INSU043': 'float64','INSU044': 'float64','INSU045': 'float64','INSU046': 'float64','INSU047': 'float64','INSU048': 'float64','INSU049': 'float64','INSU050': 'float64','INSU051': 'float64','INSU052': 'float64','INSU053': 'float64','INSU054': 'float64','INSU055': 'float64','INSU056': 'float64','INSU057': 'float64','INSU058': 'float64','INSU059': 'float64','INSU060': 'float64','INSU061': 'float64','INSU062': 'float64','INSU063': 'float64','INSU064': 'float64','INSU065': 'float64','INSU066': 'float64','INSU067': 'float64','INSU068': 'float64','INSU069': 'float64','INSU070': 'float64','INSU071': 'float64','INSU072': 'float64','INSU073': 'float64','INSU074': 'float64','INSU075': 'float64','INSU076': 'float64','INSU077': 'float64','INSU078': 'float64','INSU079': 'float64','INSU080': 'float64','INSU081': 'float64','INSU082': 'float64','INSU083': 'float64','INSU084': 'float64','INSU085': 'float64','INSU086': 'float64','INSU087': 'float64','INSU088': 'float64','INSU089': 'float64','INSU090': 'float64','INSU091': 'float64','INSU092': 'float64','INSU093': 'float64','INSU094': 'float64','INSU095': 'float64','INSU096': 'float64','INSU097': 'float64','INSU098': 'float64','INSU099': 'float64','INSU100': 'float64','INSU101': 'float64','INSU102': 'float64','INSU103': 'float64','INSU104': 'float64','INSU105': 'float64','INSU106': 'float64','INSU107': 'float64','INSU108': 'float64','INSU109': 'float64','INSU110': 'float64','INSU111': 'float64','INSU112': 'float64','INSU113': 'float64','INSU114': 'float64','INSU115': 'float64','INSU116': 'float64','INSU117': 'float64','INSU118': 'float64','INSU119': 'float64','INSU120': 'float64','INSU121': 'float64','INSU122': 'float64','INSU123': 'float64','INSU124': 'float64','INSU125': 'float64','INSU126': 'float64','INSU127': 'float64','INSU128': 'float64','INSU129': 'float64','INSU130': 'float64','INSU131': 'float64','INSU132': 'float64','INSU133': 'float64','INSU134': 'float64','INSU135': 'float64','INSU136': 'float64','INSU137': 'float64','INSU138': 'float64','INSU139': 'float64','INSU140': 'float64','INSU141': 'float64','INSU142': 'float64','INSU143': 'float64','INSU144': 'float64','INSU145': 'float64','INSU146': 'float64','INSU147': 'float64','INSU148': 'float64','INSU149': 'float64','INSU150': 'float64','INSU151': 'float64','INSU152': 'float64','INSU153': 'float64','INSU154': 'float64','INSU155': 'float64','INSU156': 'float64','UPLOAD': 'str'}
AC05_header_names=list(AC05_header_dtypes.keys())

Python version: 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
No: of logical CPU cores available: 8


In [4]:
## Selectting user and adding filepaths
user='leo' #(user variable options: 'leo','nick')
if(user=='leo'):
    base_dir='C:/Users/cnlp/Research Fellowship/'
elif(user=='nick'):
    base_dir='/Volumes/Pegasus32 R6/CreditSafe 2019 Zipped/'

os.makedirs(base_dir+'UKLTD_Database', exist_ok=True)
dir_list_file=base_dir+'/UKLTD_Scripts/processed_dir_list_AC05.txt'
database_file_folder=base_dir+'UKLTD_Database/AC05/'

#### Checking new entries

In [5]:
## Checking for new download
with open(dir_list_file, 'a+') as fd:
    fd.seek(0)
    dir_list_old=fd.read().split('\n')

print('List of processed directores:',*dir_list_old,sep='\n')
dir_list_new=[dir_name for dir_name in os.listdir(base_dir) if (dir_name.startswith("UKLTD_W") or dir_name.startswith("UKLTD_R"))]
entry_dir_list=[entry_dir for entry_dir in dir_list_new if entry_dir not in dir_list_old ]
print('\nNew entries detected:',*entry_dir_list,sep='\n')

List of processed directores:


New entries detected:
UKLTD_W_20190602
UKLTD_W_20190609
UKLTD_W_20190616
UKLTD_W_7


#### Creating/Loading database 

In [6]:
## If databse is in .csv format 
if (os.path.exists(database_file_folder)):
    print('Database found.','Reading database! \n') 
    start = time.process_time()
    with ProgressBar():
        df_database = dd.read_parquet(database_file_folder) 
    print('Time taken to read database {}:'.format(database_file_folder),time.process_time() - start,'s')
    database_missing=False
else:
    print('Database not found.','Creating database! \n')
    database_missing=True
    


Database not found. Creating database! 



#### Updating database

Rules:

1. Concatenate new entries to the existig database
2. Run de_duplicating fucntion

In [7]:
def update_database(entry_dir): 
    global df_database
    global database_missing
    
    
    ## Checking and unzipping new entry file
    entry_file="AC05_"+(entry_dir.split('_')[-2])+"_"+(entry_dir.split('_')[-1])+".txt"
    entry_file_zip="AC05_"+(entry_dir.split('_')[-2])+"_"+(entry_dir.split('_')[-1])+".rar"
    print('Unzipping:',entry_file_zip)
    try:
        pyunpack.Archive(base_dir+'{}/{}'.format(entry_dir,entry_file_zip)).extractall(base_dir+'{}/'.format(entry_dir))
        print('Entry file unzipped as:',entry_file)
    except:
        print("Rar file not found: {}".format(entry_file_zip))
        print('Skipping this entry:%s'%entry_dir)
        return None
       
    ## Reading new entry file
    start = time.process_time()
    with ProgressBar():
        df_entry_file=dd.read_csv(base_dir+entry_dir+'/'+entry_file,sep='|',names=AC05_header_names,dtype=AC05_header_dtypes,encoding='iso-8859-1',quoting=csv.QUOTE_NONE)
    print('Time taken to read:',time.process_time() - start,'s')
    df_entry_file['UPLOAD']=entry_file.split('.')[0]
    
    
    ## Adding new entry_rows to main database after checking
    if(database_missing):
        df_database=df_entry_file
        database_missing=False
    else:
        print('Appending...!')
        start = time.process_time()
        with ProgressBar():
            df_database=df_database.append(df_entry_file)
        print('Time taken to append:',time.process_time() - start,'s')  

In [8]:
## Reading from new directory and adding to database
for entry_dir in entry_dir_list:
    print('\nReading from entry dir:',entry_dir)
    update_database(entry_dir)


Reading from entry dir: UKLTD_W_20190602
Unzipping: AC05_W_20190602.rar
Entry file unzipped as: AC05_W_20190602.txt
Time taken to read: 0.015625 s

Reading from entry dir: UKLTD_W_20190609
Unzipping: AC05_W_20190609.rar
Entry file unzipped as: AC05_W_20190609.txt
Time taken to read: 0.015625 s
Appending...!
Time taken to append: 0.0625 s

Reading from entry dir: UKLTD_W_20190616
Unzipping: AC05_W_20190616.rar
Entry file unzipped as: AC05_W_20190616.txt
Time taken to read: 0.015625 s
Appending...!
Time taken to append: 0.09375 s

Reading from entry dir: UKLTD_W_7
Unzipping: AC05_W_7.rar
Rar file not found: AC05_W_7.rar
Skipping this entry:UKLTD_W_7


In [9]:
## Deduplication list 
AC05_header_names_dedup_list=list(np.array(df_database.columns))
AC05_header_names_dedup_list.remove('UPLOAD')
print('Original colomn list:',df_database.columns,'\n',)
print('Colomns to check while de-duplicating:',AC05_header_names_dedup_list)

Original colomn list: Index(['AC05', 'REGNUM', 'INSU001', 'INSU002', 'INSU003', 'INSU004', 'INSU005',
       'INSU006', 'INSU007', 'INSU008',
       ...
       'INSU148', 'INSU149', 'INSU150', 'INSU151', 'INSU152', 'INSU153',
       'INSU154', 'INSU155', 'INSU156', 'UPLOAD'],
      dtype='object', length=159) 

Colomns to check while de-duplicating: ['AC05', 'REGNUM', 'INSU001', 'INSU002', 'INSU003', 'INSU004', 'INSU005', 'INSU006', 'INSU007', 'INSU008', 'INSU009', 'INSU010', 'INSU011', 'INSU012', 'INSU013', 'INSU014', 'INSU015', 'INSU016', 'INSU017', 'INSU018', 'INSU019', 'INSU020', 'INSU021', 'INSU022', 'INSU023', 'INSU024', 'INSU025', 'INSU026', 'INSU027', 'INSU028', 'INSU029', 'INSU030', 'INSU031', 'INSU032', 'INSU033', 'INSU034', 'INSU035', 'INSU036', 'INSU037', 'INSU038', 'INSU039', 'INSU040', 'INSU041', 'INSU042', 'INSU043', 'INSU044', 'INSU045', 'INSU046', 'INSU047', 'INSU048', 'INSU049', 'INSU050', 'INSU051', 'INSU052', 'INSU053', 'INSU054', 'INSU055', 'INSU056', 'INSU057', 'I

In [10]:
## De duplicating
start = time.process_time()
df_database=df_database.drop_duplicates(subset=AC05_header_names_dedup_list).repartition(npartitions=df_database.npartitions) 
print('Time taken to de duplicate:',time.process_time() - start,'s')

Time taken to de duplicate: 0.03125 s


In [11]:
## Reseting index
df_database=df_database.reset_index(drop=True)
print('No: of partitions to be written:',df_database.npartitions)

No: of partitions to be written: 3


#### Writing into disk

In [12]:
## Writing updated databse to file
import pyarrow as pa
#schema=pa.schema([('AC05',pa.string()),('PNR',pa.float64()),('DTTITLE',pa.string()),('PTITLE',pa.string()),('INILS',pa.string()),('SUFF',pa.string()),('HNRS',pa.string()),('FNAME',pa.string()),('MNAME',pa.string()),('SNAME',pa.string()),('ADD1',pa.string()),('ADD2',pa.string()),('ADD3',pa.string()),('ADD4',pa.string()),('ADD5',pa.string()),('PCODE',pa.string()),('CNTRY',pa.string()),('BDATE',pa.string()),('FBDATE',pa.string()),('NALTY',pa.string()),('SADD',pa.string()),('MMARK',pa.string()),('URC',pa.string()),('CHGL',pa.float64()),('CHDATE',pa.string()),('RCDATE',pa.string()),('UPLOAD',pa.string())])
start = time.process_time()
with ProgressBar():
    df_database.to_parquet(database_file_folder)#,schema=schema)
print('Time taken to write:',time.process_time() - start,'s')

[########################################] | 100% Completed |  0.3s
Time taken to write: 0.421875 s


In [13]:
## Update processed directory list
print('Processed list updated')
with open(dir_list_file, 'w') as fd:
    fd.write('\n'.join(dir_list_old+entry_dir_list))

Processed list updated
